# IMPORT LIBRARY

In [1]:
!pip install underthesea
!pip install sentencepiece
!pip install transformers==4.44.2
!pip install einops
!pip install einops_exts
!pip install pandas
!pip install torch==2.4.0
!pip install torchvision==0.19.0 
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 60.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 55.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 67.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.4 MB/s eta 0:00:

In [2]:
from underthesea import word_tokenize,text_normalize
import torch
from transformers import AutoTokenizer,get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
import pandas as pd
import json
import numpy as np
from PIL import Image
from torchvision import transforms
import os
import re
import sentencepiece

In [3]:
from transformers import AutoFeatureExtractor,CLIPModel, CLIPProcessor, AutoTokenizer, AutoModel,RobertaModel, AutoImageProcessor
from transformers import CLIPVisionModel, CLIPVisionConfig

import torch
from torchvision.io import read_image
import open_clip

In [4]:
sss = 18 + 19
torch.manual_seed(sss)
np.random.seed(sss)

# TEXT PRE-PROCESSING

In [5]:
def convert_unicode(text):
  char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
  charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
  char1252 = char1252.split('|')
  charutf8 = charutf8.split('|')

  dic = {}
  for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
  return re.sub(
      r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
      lambda x: dic[x.group()], text
)


In [6]:
import unicodedata

class TextNormalize:
    def __init__(self):
        self.vowels_to_ids = {}
        self.vowels_table = [
            ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
            ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
            ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
            ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
            ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
            ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
            ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
            ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'o'],
            ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
            ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
            ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
            ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
        ]
        pass

    def createVowelsTable(self):
        """Create Vowels Table"""
        for i in range(len(self.vowels_table)):
            for j in range(len(self.vowels_table[i]) - 1):
                self.vowels_to_ids[self.vowels_table[i][j]] = (i, j)

    def IsValidVietnameseWord(self,word):
        """Nguyên âm chỉ có thể đứng chung với nguyên âm. Một từ không thể có 2 nguyên âm cách nhau bởi 1 phụ âm"""
        chars = list(word)
        #nguyen am
        vowel_index = -1
        for i in range(len(chars)):
            idx_vowel_table = self.vowels_to_ids.get(chars[i],(-1,-1))[0]
            if idx_vowel_table != -1:
                if vowel_index == -1:
                    vowel_index = i
                else:
                    if i - vowel_index != 1:
                        return False
                    vowel_index = i
        return True

    def WordStandardized(self,word):
        """Standardize Word"""
        if not self.IsValidVietnameseWord(word):
            return word

        chars = list(word)
        vowel_indexes = []

        # tìm vị trí nguyên âm
        qu_or_gi = False
        thanh_dieu = 0
        for i in range(len(chars)):
            vowel_table_row, vowel_table_col = self.vowels_to_ids.get(chars[i],(-1,-1))
            if vowel_table_row == -1 :
                continue
            # qu
            if vowel_table_row == 9:
                if i != 0 and chars[i-1] == 'q':
                    chars[i] = 'u'
                    qu_or_gi = True
            # gi
            elif vowel_table_row == 5:
                if i != 0 and chars[i-1] == 'g':
                    chars[i] = 'i'
                    qu_or_gi = True

            # có chứa thanh điệu
            if vowel_table_col != 0:
                thanh_dieu = vowel_table_col
                chars[i] = self.vowels_table[vowel_table_row][0]

            vowel_indexes.append(i)
        # 1 nguyên âm
        if len(vowel_indexes) == 1:
            c = chars[vowel_indexes[0]]
            chars[vowel_indexes[0]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
            return ''.join(chars)

        for idx_vowel in vowel_indexes:
            vowel_table_row, vowel_table_col = self.vowels_to_ids.get(chars[idx_vowel],(-1,-1))
            #ê, ơ, ô
            if vowel_table_row == 4 or vowel_table_row == 7 or vowel_table_row == 8:
                c = chars[idx_vowel]
                chars[idx_vowel] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            # kiểm tra qu và gi, 2-3 nguyên âm thì nguyên âm thứ 2 chứa dấu
            if qu_or_gi:
                if len(vowel_indexes) == 2 or len(vowel_indexes) == 3:
                    c = chars[vowel_indexes[1]]
                    chars[vowel_indexes[1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            # 2 nguyên âm
            if len(vowel_indexes) == 2:
                # âm cuối là nguyên âm
                if vowel_indexes[-1] == len(chars) - 1:
                    c = chars[vowel_indexes[0]]
                    chars[vowel_indexes[0]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                else:
                    c = chars[vowel_indexes[-1]]
                    chars[vowel_indexes[-1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

            elif len(vowel_indexes) == 3:
                # âm cuối là nguyên âm
                if vowel_indexes[-1] == len(chars) - 1:
                    c = chars[vowel_indexes[1]]
                    chars[vowel_indexes[1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                else:
                    c = chars[vowel_indexes[-1]]
                    chars[vowel_indexes[-1]] = self.vowels_table[self.vowels_to_ids[c][0]][thanh_dieu]
                return ''.join(chars)

        return ''.join(chars)

    def normalize(self,text):


        #Chuyen sang viet thuong
        text = text.lower()
        text = unicodedata.normalize('NFC', text)

        # Rút gọn từ kéo dài
        text = re.sub(r'(\W)\1+',r'\1',text)

        # xóa các emoji dư thừa
#         emoji_pattern = re.compile(
#             "["
#             "\U0001F600-\U0001F64F"  # Emoticons
#             "\U0001F300-\U0001F5FF"  # Symbols & pictographs
#             "\U0001F680-\U0001F6FF"  # Transport & map symbols
#             "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
#             "\U00002500-\U00002BEF"  # Chinese char
#             "\U00002702-\U000027B0"
#             "\U00002702-\U000027B0"
#             "\U000024C2-\U0001F251"
#             "\U0001f926-\U0001f937"
#             "\U00010000-\U0010ffff"
#             "]+",
#             flags=re.UNICODE,
#         )
#         text = emoji_pattern.sub(r'',text) # no emoji
        # remove hastag
        text = re.sub("(@[A-Za-z0-9]+)|(#[0-9A-Za-z]+)","", text)

        # xóa space d
        text = re.sub(r"( )\1+",r'\1',text)
#         text = re.sub(r"[:)^@!`~%;?(\+\-\'\"]+",r'',text)
        text = text.replace("“","")
        text = text.replace("\n", " ")
        text = " ".join(text.split())
        return text

In [7]:
%ls

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split 

data = pd.read_json('/kaggle/input/train-dsc2024/vimmsd-train.json',orient = 'index').reset_index()
# data = data[data['label'] != 'not-sarcasm']

In [9]:
train_data, dev_data = train_test_split(data, test_size=0.1, stratify=data['label'], random_state = sss) 
train_data = train_data.reset_index() 
dev_data = dev_data.reset_index()

In [10]:
normalize_class = TextNormalize()

In [11]:
train_data['caption'] = train_data['caption'].apply(lambda x: text_normalize(normalize_class.normalize(x)))
dev_data['caption'] = dev_data['caption'].apply(lambda x: text_normalize(normalize_class.normalize(x)))

In [12]:
train_data

,level_0,index,image,caption,label
0,9090,9090,110f61e365393bbd6234d89dec7adde94784b22ebf45d2...,hạt dẻ cười = ),multi-sarcasm
1,898,898,aba83dfcaf0b3b515ce2d9663a062538ca257a6f99c8d4...,nếu cây ô-rô ( ilex aquifolium ) phát hiện lá ...,not-sarcasm
2,8124,8124,1d55a3f21394857c538ee4df518ee03254570a914243a4...,_f33 - bức xúc anh em xã hội quanh co đổ tội c...,multi-sarcasm
3,210,210,3ca63743fa3c553b31e6dbde5884196019ae0d22899dd4...,nó lại hợp lý 🤣,multi-sarcasm
4,9652,9652,ce429a6a3e9ee68c6df95e9cda6619ebb22c217c37f4ac...,tính dụ fan arsenal xem đến hết trận hay gì = ),multi-sarcasm
...,...,...,...,...,...
9719,8305,8305,62c96968a00ddb87ef8e60505d8d1182084f4057781483...,🌟 một từ thôi : hay !,not-sarcasm
9720,10529,10529,d2b63d2ab3391166b4bfb71b7fbd20be617940bbe188be...,"lâm minh cho biết sau khi bình tĩnh , cô không...",not-sarcasm
9721,4738,4738,94c6adcbd21f4dfc744ac059a61476a03ff775672d2e82...,bắt gặp một rapper cộm cán trong làng rap úc ....,multi-sarcasm
9722,6795,6795,618f7a296eedf5bd1ca491dbdcb465675648735bd32c2a...,đơn vị sản xuất series đu đêm metub cùng hoa h...,not-sarcasm


In [13]:
class ViMMDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, img_folder, vision_processor, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.img_folder = img_folder
        self.vision_processor = vision_processor
        self.labels = ['not-sarcasm', 'image-sarcasm','text-sarcasm','multi-sarcasm']

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        text = self.data.loc[idx,'caption']
        img_name = self.data.loc[idx,'image']
        img_file = os.path.join(self.img_folder,img_name)
        label = self.data.loc[idx,'label']
        label = self.labels.index(label)
        
        tokenizer_seq_dict = self.tokenizer("<image>", text, max_length=self.max_len,truncation=True,padding='max_length', return_length=True, return_tensors='pt')
        input_ids = tokenizer_seq_dict['input_ids'][0]
        attention_mask = tokenizer_seq_dict['attention_mask']

        # feature_file = os.path.join('dino_large_features/dino_large_features/' + img_name.split('.')[0] + '.npy')
        # features = np.load(feature_file, allow_pickle=True)[()]

        # pixel_values = features['pixel_values']

        image = Image.open(img_file)        
        pixel_values = self.vision_processor(image).unsqueeze(0).unsqueeze(0)

        return input_ids, attention_mask, pixel_values, torch.tensor(label)

In [14]:
PRETRAINED_PATH = "xlm-roberta-base"
VISION_PRETRAINED_PATH = 'facebook/dinov2-large'

In [15]:
vision_processor = AutoImageProcessor.from_pretrained(VISION_PRETRAINED_PATH)

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

In [16]:
_, _, vision_processor = open_clip.create_model_and_transforms(
    'ViT-B-16',
    pretrained='openai'
)

100%|███████████████████████████████████████| 351M/351M [00:10<00:00, 32.8MiB/s]


In [17]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_PATH)
tokenizer.add_special_tokens(
        {"additional_special_tokens": ["<image>"]}
    )


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1

In [18]:
train_set = ViMMDataset(train_data, tokenizer, '/kaggle/input/train-dsc2024/training-images/train-images', vision_processor, max_len = 200)
dev_set = ViMMDataset(dev_data, tokenizer, '/kaggle/input/train-dsc2024/training-images/train-images', vision_processor, max_len = 200)

In [19]:
# for i in range(len(train_set)):
#     try:
#         t = train_set[i]
#     except:
#         print(i)

In [20]:
aa = 0
for i in train_set[aa][0]:
    print(tokenizer.convert_ids_to_tokens(i.item()),end=' ' )
print()

<s> <image> </s> </s> ▁hạt ▁d ẻ ▁cười ▁= ▁) </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [21]:
from tqdm import tqdm
y = []
for i in tqdm(range(len(train_set))):
  y.append(train_set[i][-1].item())


100%|██████████| 9724/9724 [03:20<00:00, 48.41it/s]


In [22]:
y[:20]

[3, 0, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 0, 3, 0]

In [23]:
import numpy as np
from torch.utils.data.sampler import WeightedRandomSampler

counts = np.bincount(y)
labels_weights = 1. / counts
# labels_weights[-1] = labels_weights[-1]
weights = labels_weights[y]
print(counts,labels_weights)

[5456  398   69 3801] [0.00018328 0.00251256 0.01449275 0.00026309]


In [24]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 4, shuffle = False)
dev_loader = torch.utils.data.DataLoader(dev_set, batch_size = 4)

# MODEL

In [25]:
import torch
import torch.nn as nn
import copy
import numpy as np
import math
import torch.nn.functional as F
from transformers import AutoModel
from torch.autograd import Variable


In [26]:
import torch
from einops import rearrange, repeat
from einops_exts import rearrange_many
from torch import einsum, nn

In [27]:
import os
mode = 'FP16'
os.environ["TORCH_CUDNN_V8_API_ENABLED"] = "1"
torch.backends.cuda.matmul.allow_tf32 = True if mode == 'TF32' else False
scaler = torch.cuda.amp.GradScaler(enabled=True if mode == 'FP16' else False)


/tmp/ipykernel_30/3556953468.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True if mode == 'FP16' else False)


In [28]:
def exists(val):
    return val is not None

def FeedForward(dim, mult=4):
    inner_dim = int(dim * mult)
    return nn.Sequential(
        nn.LayerNorm(dim),
        nn.Linear(dim, inner_dim, bias=False),
        nn.GELU(),
        nn.Linear(inner_dim, dim, bias=False),
    )

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [30]:
class PerceiverAttention(nn.Module):
    def __init__(self, *, dim, dim_head=64, heads=8):
        super().__init__()
        self.scale = dim_head**-0.5
        self.heads = heads
        inner_dim = dim_head * heads

        self.norm_media = nn.LayerNorm(dim)
        self.norm_latents = nn.LayerNorm(dim)

        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias=False)
        self.to_out = nn.Linear(inner_dim, dim, bias=False)

    def forward(self, x, latents):
        """
        Args:
            x (torch.Tensor): image features
                shape (b, T, n1, D)
            latent (torch.Tensor): latent features
                shape (b, T, n2, D)
        """
        x = self.norm_media(x)
        latents = self.norm_latents(latents)

        h = self.heads

        q = self.to_q(latents)
        kv_input = torch.cat((x, latents), dim=-2)
        k, v = self.to_kv(kv_input).chunk(2, dim=-1)
        q, k, v = rearrange_many((q, k, v), "b t n (h d) -> b h t n d", h=h)
        q = q * self.scale

        # attention
        sim = einsum("... i d, ... j d  -> ... i j", q, k)
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)

        out = einsum("... i j, ... j d -> ... i d", attn, v)
        out = rearrange(out, "b h t n d -> b t n (h d)", h=h)
        return self.to_out(out)

In [31]:
class PerceiverResampler(nn.Module):
    def __init__(
        self,
        *,
        dim,
        depth=6,
        dim_head=64,
        heads=8,
        num_latents=64,
        max_num_media=None,
        max_num_frames=None,
        ff_mult=4,
    ):
        super().__init__()
        self.latents = nn.Parameter(torch.randn(num_latents, dim))
        self.frame_embs = (
            nn.Parameter(torch.randn(max_num_frames, dim))
            if exists(max_num_frames)
            else None
        )
        self.media_time_embs = (
            nn.Parameter(torch.randn(max_num_media, 1, dim))
            if exists(max_num_media)
            else None
        )

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(
                nn.ModuleList(
                    [
                        PerceiverAttention(dim=dim, dim_head=dim_head, heads=heads),
                        FeedForward(dim=dim, mult=ff_mult),
                    ]
                )
            )

        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): image features
                shape (b, T, F, v, D)
        Returns:
            shape (b, T, n, D) where n is self.num_latents
        """
        b, T, F, v = x.shape[:4]

        # frame and media time embeddings
        if exists(self.frame_embs):
            frame_embs = repeat(self.frame_embs[:F], "F d -> b T F v d", b=b, T=T, v=v)
            x = x + frame_embs
        x = rearrange(
            x, "b T F v d -> b T (F v) d"
        )  # flatten the frame and spatial dimensions
        if exists(self.media_time_embs):
            x = x + self.media_time_embs[:T]

        # blocks
        latents = repeat(self.latents, "n d -> b T n d", b=b, T=T)
        for attn, ff in self.layers:
            latents = attn(x, latents) + latents
            latents = ff(latents) + latents
        return self.norm(latents)

In [32]:
# gated cross attention
class MaskedCrossAttention(nn.Module):
    def __init__(
        self,
        *,
        dim,
        dim_visual,
        dim_head=64,
        heads=8,
        only_attend_immediate_media=True,
    ):
        super().__init__()
        self.scale = dim_head**-0.5
        self.heads = heads
        inner_dim = dim_head * heads

        self.norm = nn.LayerNorm(dim)

        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(dim_visual, inner_dim * 2, bias=False)
        self.to_out = nn.Linear(inner_dim, dim, bias=False)

        # whether for text to only attend to immediate preceding image, or all previous images
        self.only_attend_immediate_media = only_attend_immediate_media

    def forward(self, x, media, media_locations=None, use_cached_media=False):
        """
        Args:
            x (torch.Tensor): text features
                shape (B, T_txt, D_txt)
            media (torch.Tensor): image features
                shape (B, T_img, n, D_img) where n is the dim of the latents
            media_locations: boolean mask identifying the media tokens in x
                shape (B, T_txt)
            use_cached_media: bool
                If true, treat all of x as if they occur after the last media
                registered in media_locations. T_txt does not need to exactly
                equal media_locations.shape[1] in this case
        """

        if not use_cached_media:
            assert (
                media_locations.shape[1] == x.shape[1]
            ), f"media_location.shape is {media_locations.shape} but x.shape is {x.shape}"

        T_txt = x.shape[1]
        _, T_img, n = media.shape[:3]
        h = self.heads

        x = self.norm(x)

        q = self.to_q(x)
        media = rearrange(media, "b t n d -> b (t n) d")

        k, v = self.to_kv(media).chunk(2, dim=-1)
        q, k, v = rearrange_many((q, k, v), "b n (h d) -> b h n d", h=h)

        q = q * self.scale

        sim = einsum("... i d, ... j d -> ... i j", q, k)

        if exists(media_locations):
            media_time = torch.arange(T_img, device=x.device) + 1

            if use_cached_media:
                # text time is set to the last cached media location
                text_time = repeat(
                    torch.count_nonzero(media_locations, dim=1),
                    "b -> b i",
                    i=T_txt,
                )
            else:
                # at each boolean of True, increment the time counter (relative to media time)
                text_time = media_locations.cumsum(dim=-1)

            # text time must equal media time if only attending to most immediate image
            # otherwise, as long as text time is greater than media time (if attending to all previous images / media)
            mask_op = torch.eq if self.only_attend_immediate_media else torch.ge

            text_to_media_mask = mask_op(
                rearrange(text_time, "b i -> b 1 i 1"),
                repeat(media_time, "j -> 1 1 1 (j n)", n=n),
            )
            sim = sim.masked_fill(~text_to_media_mask, -torch.finfo(sim.dtype).max)

        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)

        if exists(media_locations) and self.only_attend_immediate_media:
            # any text without a preceding media needs to have attention zeroed out
            text_without_media_mask = text_time == 0
            text_without_media_mask = rearrange(
                text_without_media_mask, "b i -> b 1 i 1"
            )
            attn = attn.masked_fill(text_without_media_mask, 0.0)

        out = einsum("... i j, ... j d -> ... i d", attn, v)
        out = rearrange(out, "b h n d -> b n (h d)")
        return self.to_out(out)

In [33]:
class GatedCrossAttentionBlock(nn.Module):
    def __init__(
        self,
        *,
        dim,
        dim_visual,
        dim_head=64,
        heads=8,
        ff_mult=4,
        only_attend_immediate_media=True,
    ):
        super().__init__()
        self.attn = MaskedCrossAttention(
            dim=dim,
            dim_visual=dim_visual,
            dim_head=dim_head,
            heads=heads,
            only_attend_immediate_media=only_attend_immediate_media,
        )
        self.attn_gate = nn.Parameter(torch.tensor([0.0]))

        self.ff = FeedForward(dim, mult=ff_mult)
        self.ff_gate = nn.Parameter(torch.tensor([0.0]))

    def forward(
        self,
        x,
        media,
        media_locations=None,
        use_cached_media=False,
    ):
        x = (
            self.attn(
                x,
                media,
                media_locations=media_locations,
                use_cached_media=use_cached_media,
            )
            * self.attn_gate.tanh()
            + x
        )
        x = self.ff(x) * self.ff_gate.tanh() + x

        return x

In [34]:
def apply_with_stopping_condition(
    module, apply_fn, apply_condition=None, stopping_condition=None, **other_args
):
    if stopping_condition(module):
        return
    if apply_condition(module):
        apply_fn(module, **other_args)
    for child in module.children():
        apply_with_stopping_condition(
            child,
            apply_fn,
            apply_condition=apply_condition,
            stopping_condition=stopping_condition,
            **other_args
        )

In [35]:
def extend_instance(obj, mixin):
    """Apply mixins to a class instance after creation"""
    base_cls = obj.__class__
    base_cls_name = obj.__class__.__name__
    obj.__class__ = type(
        base_cls_name, (mixin, base_cls), {}
    )  # mixin needs to go first for our forward() logic to work


def getattr_recursive(obj, att):
    """
    Return nested attribute of obj
    Example: getattr_recursive(obj, 'a.b.c') is equivalent to obj.a.b.c
    """
    if att == "":
        return obj
    i = att.find(".")
    if i < 0:
        return getattr(obj, att)
    else:
        return getattr_recursive(getattr(obj, att[:i]), att[i + 1 :])


def setattr_recursive(obj, att, val):
    """
    Set nested attribute of obj
    Example: setattr_recursive(obj, 'a.b.c', val) is equivalent to obj.a.b.c = val
    """
    if "." in att:
        obj = getattr_recursive(obj, ".".join(att.split(".")[:-1]))
    setattr(obj, att.split(".")[-1], val)



In [36]:
class FlamingoLayer(nn.Module):
    """
    FlamingoLayer is a wrapper around the GatedCrossAttentionBlock and DecoderLayer.
    """

    def __init__(
        self, gated_cross_attn_layer, encoder_layer, gradient_checkpointing=False
    ):
        super().__init__()
        self.gated_cross_attn_layer = gated_cross_attn_layer
        self.vis_x = None
        self.encoder_layer = encoder_layer
        self.media_locations = None
        if self.gated_cross_attn_layer is not None:
            self.gated_cross_attn_layer._use_gradient_checkpointing = (
                gradient_checkpointing
            )

    def is_conditioned(self) -> bool:
        """Check whether the layer is conditioned."""
        return self.vis_x is not None and self.media_locations is not None

    # Used this great idea from this implementation of Flamingo (https://github.com/dhansmair/flamingo-mini/)
    def condition_vis_x(self, vis_x):
        self.vis_x = vis_x

    def condition_media_locations(self, media_locations):
        self.media_locations = media_locations

    def condition_use_cached_media(self, use_cached_media):
        self.use_cached_media = use_cached_media

    def forward(
        self,
        lang_x, 
        attention_mask,
        *args,
        **encoder_layer_kwargs
    ):
        # Cross attention
        if self.gated_cross_attn_layer is not None:
            if self.vis_x is None:
                raise ValueError("vis_x must be conditioned before forward pass")

            if self.media_locations is None:
                raise ValueError(
                    "media_locations must be conditioned before forward pass"
                )

            lang_x = self.gated_cross_attn_layer(
                lang_x,
                self.vis_x,
                media_locations=self.media_locations,
                use_cached_media=self.use_cached_media,
            )

        # Normal decoder layer
        lang_x = self.encoder_layer(
            lang_x, attention_mask, *args
        )
        return lang_x

In [37]:
from transformers import AutoFeatureExtractor,CLIPModel, CLIPProcessor, AutoTokenizer, AutoModel,RobertaModel

class FlamingoLMMixin(nn.Module):
    """
    Mixin to add cross-attention layers to a language model.
    """

    def set_encoder_layers_attr_name(self, encoder_layers_attr_name):
        self.encoder_layers_attr_name = encoder_layers_attr_name

    def _get_encoder_layers(self):
        return getattr_recursive(self, self.encoder_layers_attr_name)

    def _set_encoder_layers(self, value):
        setattr_recursive(self, self.encoder_layers_attr_name, value)

    def init_flamingo(
        self,
        media_token_id,
        lang_hidden_size,
        vis_hidden_size,
        cross_attn_every_n_layers,
        gradient_checkpointing,
    ):
        """
        Initialize Flamingo by adding a new gated cross attn to the decoder. Store the media token id for computing the media locations.
        """
        self.old_encoder_blocks = self._get_encoder_layers()
        self.gated_cross_attn_layers = nn.ModuleList(
            [
                GatedCrossAttentionBlock(
                    dim=lang_hidden_size, dim_visual=vis_hidden_size
                )
                if (layer_idx + 1) % cross_attn_every_n_layers == 0
                else None
                for layer_idx, _ in enumerate(self._get_encoder_layers())
            ]
        )
        self.init_flamingo_layers(gradient_checkpointing)
        self.media_token_id = media_token_id
        self.initialized_flamingo = True
        self._use_cached_vision_x = False

    def init_flamingo_layers(self, gradient_checkpointing):
        """
        Re initializes the FlamingoLayers.
        Propagates any changes made to self.gated_corss_attn_layers or self.old_encoder_blocks
        """
        self._set_encoder_layers(
            nn.ModuleList(
                [
                    FlamingoLayer(
                        gated_cross_attn_layer, encoder_layer, gradient_checkpointing
                    )
                    for gated_cross_attn_layer, encoder_layer in zip(
                        self.gated_cross_attn_layers, self.old_encoder_blocks
                    )
                ]
            )
        )

    def forward(self, input_ids,attention_mask, **kwargs):

        """Condition the Flamingo layers on the media locations before forward()"""
        if not self.initialized_flamingo:
            raise ValueError(
                "Flamingo layers are not initialized. Please call `init_flamingo` first."
            )
        media_locations = input_ids == self.media_token_id

        # if there are media already cached and we're generating and there are no media tokens in the input,
        # we'll assume that ALL input tokens should attend to the last previous media that is cached.
        # this is especially important for HF generate() compatibility, since generate() calls forward()
        # repeatedly one token at a time (with no media tokens).
        # without this check, the model would not attend to any images when generating (after the first token)
        use_cached_media_locations = (
            self._use_cached_vision_x
            and self.is_conditioned()
            and not media_locations.any()
        )

        for layer in self._get_encoder_layers():
            if not use_cached_media_locations:
                layer.condition_media_locations(media_locations)
            layer.condition_use_cached_media(use_cached_media_locations)

        # package arguments for the other parent's forward. since we don't know the order of the arguments,
        # make them all kwargs

        kwargs["input_ids"] = input_ids
        kwargs["attention_mask"] = attention_mask

        return super().forward(**kwargs)  # Call the other parent's forward method

    def is_conditioned(self) -> bool:
        """Check whether all decoder layers are already conditioned."""
        return all(l.is_conditioned() for l in self._get_encoder_layers())

    def clear_conditioned_layers(self):
        for layer in self._get_encoder_layers():
            layer.condition_vis_x(None)
            layer.condition_media_locations(None)
            layer.condition_use_cached_media(None)

In [38]:
import torch
from einops import rearrange
from torch import nn
from transformers.modeling_outputs import CausalLMOutputWithPast
from torch.distributed.fsdp import (
    FullyShardedDataParallel as FSDP,
)

class Flamingo(nn.Module):
    def __init__(
        self,
        vision_encoder: nn.Module,
        lang_encoder: nn.Module,
        vis_dim: int,
        media_token_id: int,
        n_classes = 4,
        cross_attn_every_n_layers: int = 1,
        gradient_checkpointing: bool = False,
    ):
        """
        Args:
            vision_encoder (nn.Module): HF CLIPModel
            lang_encoder (nn.Module): HF causal language model
                vis_dim (int): Dimension of the visual features.
                Visual features are projected to match this shape along the last dimension.
            cross_attn_every_n_layers (int, optional): How often to apply cross attention after transformer layer. Defaults to 1.
        """
        super().__init__()

        self.media_token_id = media_token_id
        self.vis_dim = vis_dim
        if hasattr(lang_encoder.config, "d_model"):
            self.lang_dim = lang_encoder.config.d_model  # mpt uses d_model
        else:
            self.lang_dim = lang_encoder.config.hidden_size

        self.vision_encoder = vision_encoder
        self.perceiver = PerceiverResampler(dim=self.vis_dim)
        self.lang_encoder = lang_encoder
        self.lang_encoder.init_flamingo(
            media_token_id=media_token_id,
            lang_hidden_size=self.lang_dim,
            vis_hidden_size=self.vis_dim,
            cross_attn_every_n_layers=cross_attn_every_n_layers,
            gradient_checkpointing=gradient_checkpointing,
        )
        self.classifier = nn.Linear(self.lang_dim*4*2,n_classes)

        self._use_gradient_checkpointing = gradient_checkpointing
        self.perceiver._use_gradient_checkpointing = gradient_checkpointing

    def forward(
        self,
        vision_x: torch.Tensor,
        lang_x: torch.Tensor,
        attention_mask = None, 
        clear_conditioned_layers: bool = True,
        ):
        """
        Forward pass of Flamingo.

        Args:
            vision_x (torch.Tensor): Vision input
                shape (B, T_img, F, C, H, W) with F=1
            lang_x (torch.Tensor): Language input ids
                shape (B, T_txt)
            attention_mask (torch.Tensor, optional): Attention mask. Defaults to None.
            labels (torch.Tensor, optional): Labels. Defaults to None.
            clear_conditioned_layers: if True, clear the conditioned layers
                once the foward pass is completed. Set this to false if the
                same set of images will be reused in another subsequent
                forward pass.
            past_key_values: pre-computed values to pass to language model.
                See past_key_values documentation in Hugging Face
                CausalLM models.
            use_cache: whether to use cached key values. See use_cache
                documentation in Hugging Face CausalLM models.
        """
        assert (
            self.lang_encoder.initialized_flamingo
        ), "Flamingo layers are not initialized. Please call `init_flamingo` first."

        assert (
            self.lang_encoder._use_cached_vision_x or vision_x is not None
        ), "Must provide either vision_x or have precached media using cache_media()."
        
        self.lang_encoder._use_cached_vision_x = True
        self._encode_vision_x(vision_x=vision_x)
        self._condition_media_locations(input_ids=lang_x)

        output = self.lang_encoder(
            input_ids=lang_x,
            attention_mask = attention_mask,
            output_hidden_states=True
        )
        # output = output.pooler_output
        # output = self.classifier(output)

        hidden_states = output.hidden_states
        last_hidden_state = torch.cat(hidden_states[-4:], dim=-1) # B, 1 + seq_len, H*4
        text_features = last_hidden_state[:,0,:] # B, H*4
        visual_features = last_hidden_state[:,1,:] # B, H*4
        cls_and_visual = torch.cat([text_features,visual_features], dim=1)
        output = self.classifier(cls_and_visual)
        
        self.lang_encoder.clear_conditioned_layers()
        # self.lang_encoder._use_cached_vision_x = False

        return output

        
    def _encode_vision_x(self, vision_x: torch.Tensor):
        """
        Compute media tokens from vision input by passing it through vision encoder and conditioning language model.
        Args:
            vision_x (torch.Tensor): Vision input
                shape (B, T_img, F, C, H, W)
                Images in the same chunk are collated along T_img, and frames are collated along F
                Currently only F=1 is supported (single-frame videos)

        rearrange code based on https://github.com/dhansmair/flamingo-mini
        """

        assert vision_x.ndim == 6, "vision_x should be of shape (b, T_img, F, C, H, W)"
        b, T, F = vision_x.shape[:3]
        assert F == 1, "Only single frame supported"

        vision_x = rearrange(vision_x, "b T F c h w -> (b T F) c h w")
        with torch.no_grad():
            # vision_x = self.vision_encoder(vision_x)[1].unsqueeze(1) #check
            vision_x = self.vision_encoder(vision_x)[1]

        vision_x = rearrange(vision_x, "(b T F) v d -> b T F v d", b=b, T=T, F=F)
        vision_x = self.perceiver(vision_x)
        # print('vision size:', vision_x.size())
        for layer in self.lang_encoder._get_encoder_layers():
            layer.condition_vis_x(vision_x)
        def clip_grad_norm_(max_norm):
            self.perceiver.clip_grad_norm_(max_norm)
            for layer in self.lang_encoder.gated_cross_attn_layers:
                if layer is not None:
                    layer.clip_grad_norm_(max_norm)
            self.lang_encoder.get_input_embeddings().clip_grad_norm_(max_norm)

        self.clip_grad_norm_ = clip_grad_norm_

    def _condition_media_locations(self, input_ids: torch.Tensor):
        """
        Compute the media token locations from lang_x and condition the language model on these.
        Args:
            input_ids (torch.Tensor): Language input
                shape (B, T_txt)
        """
        media_locations = input_ids == self.media_token_id

        for layer in self.lang_encoder._get_encoder_layers():
            layer.condition_media_locations(media_locations)

    def cache_media(self, input_ids: torch.Tensor, vision_x: torch.Tensor):
        """
        Pre-cache a prompt/sequence of images / text for log-likelihood evaluations.
        All subsequent calls to forward() will generate attending to the LAST
        image in vision_x.
        This is not meant to be used to cache things for generate().
        Args:
            input_ids (torch.Tensor): Language input
                shape (B, T_txt)
            vision_x (torch.Tensor): Vision input
                shape (B, T_img, F, C, H, W)
                Images in the same chunk are collated along T_img, and frames are collated along F
                Currently only F=1 is supported (single-frame videos)
        """
        self._encode_vision_x(vision_x=vision_x)
        self._condition_media_locations(input_ids=input_ids)
        self.lang_encoder._use_cached_vision_x = True

    def uncache_media(self):
        """
        Clear all conditioning.
        """
        self.lang_encoder.clear_conditioned_layers()
        self.lang_encoder._use_cached_vision_x = False

In [39]:
__KNOWN_encoder_layerS_ATTR_NAMES = {
    'roberta': 'encoder.layer',
    'MultilingualCLIP': 'transformer.encoder.layer',
    'ernie': 'encoder.layers'
}

In [40]:
def _infer_encoder_layers_attr_name(model):
    for k in __KNOWN_encoder_layerS_ATTR_NAMES:
        if k.lower() in model.__class__.__name__.lower():
            return __KNOWN_encoder_layerS_ATTR_NAMES[k]

    raise ValueError(
        f"We require the attribute name for the nn.ModuleList in the decoder storing the transformer block layers. Please supply this string manually."
    )


In [41]:
from transformers import AutoFeatureExtractor,CLIPModel, CLIPProcessor, AutoTokenizer, AutoModel,RobertaModel
from transformers import CLIPVisionModel, CLIPVisionConfig

import torch
from torchvision.io import read_image

In [42]:
lang_encoder = AutoModel.from_pretrained(PRETRAINED_PATH)
# vision_encoder = AutoModel.from_pretrained(VISION_PRETRAINED_PATH)

temp_vision, _, _ = open_clip.create_model_and_transforms(
    'ViT-B-16',
    pretrained='openai'
)

vision_encoder = temp_vision.visual
vision_encoder.output_tokens = True
vision_encoder.attn_pool = vision_encoder.proj = None


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [43]:
print("trainable parameter vision:", sum(p.numel() for p in vision_encoder.parameters()))
print("trainable parameter text:", sum(p.numel() for p in lang_encoder.parameters()))


trainable parameter vision: 85799424
trainable parameter text: 278043648


In [44]:
extend_instance(lang_encoder, FlamingoLMMixin)

In [45]:
encoder_layers_attr_name  = _infer_encoder_layers_attr_name(lang_encoder)
lang_encoder.set_encoder_layers_attr_name(encoder_layers_attr_name)
lang_encoder.resize_token_embeddings(len(tokenizer))


Embedding(250003, 768, padding_idx=1)

In [46]:
model = Flamingo(
    vision_encoder,
    lang_encoder,
    cross_attn_every_n_layers=1,
    vis_dim = 768,
    media_token_id = tokenizer.convert_tokens_to_ids('<image>'),
    gradient_checkpointing=False,
    n_classes = 4
)

In [47]:
model.to(device)

Flamingo(
  (vision_encoder): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [48]:
print("trainable parameter text-vision:", sum(p.numel() for p in model.parameters()))


trainable parameter text-vision: 477229852


In [49]:
for data in train_loader:
    break
    
with torch.no_grad():
    input_ids, attention_mask, pixel_values, label = data
    pixel_values = pixel_values.float()
    
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    pixel_values = pixel_values.to(device)
    label = label.to(device)
    
    logits = model(
            vision_x =  pixel_values,
            lang_x =  input_ids,
            attention_mask = attention_mask
            )
logits

tensor([[-0.9241, -0.6584,  0.3598,  0.5388],
        [-0.8011, -0.6974,  0.3471,  0.5763],
        [-0.8524, -0.7400,  0.3725,  0.6185],
        [-0.9255, -0.6789,  0.3274,  0.5100]], device='cuda:0')

# TRAINING

In [50]:
import math
import torch
from torch.optim import Optimizer

class AdEMAMix(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999, 0.9999), eps=1e-8,
                 weight_decay=0, alpha=5.0, T_alpha_beta3=None):
        if not 0.0 <= lr:
            raise ValueError(f"Invalid learning rate: {lr}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon value: {eps}")
        assert len(betas) == 3, f"Invalid beta parameters: {betas}, expected 3"
        assert all(0.0 <= beta < 1.0 for beta in betas), f"Invalid beta parameters: {betas}"
        if not 0.0 <= weight_decay:
            raise ValueError(f"Invalid weight_decay value: {weight_decay}")
        
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay,
                        alpha=alpha, T_alpha_beta3=T_alpha_beta3)
        super(AdEMAMix, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdEMAMix, self).__setstate__(state)

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            params_with_grad = []
            grads = []
            exp_avgs = []
            exp_avg_sqs = []
            exp_avg_slow = []
            state_steps = []

            for p in group['params']:
                if p.grad is not None:
                    params_with_grad.append(p)
                    if p.grad.is_sparse:
                        raise RuntimeError('AdEMAMix does not support sparse gradients')
                    grads.append(p.grad)

                    state = self.state[p]
                    # Lazy state initialization
                    if len(state) == 0:
                        state['step'] = 0
                        # Exponential moving average of gradient values
                        state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                        # Exponential moving average of squared gradient values
                        state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                        # Slow exponential moving average
                        state['exp_avg_slow'] = torch.zeros_like(p, memory_format=torch.preserve_format)

                    exp_avgs.append(state['exp_avg'])
                    exp_avg_sqs.append(state['exp_avg_sq'])
                    exp_avg_slow.append(state['exp_avg_slow'])
                    state['step'] += 1
                    state_steps.append(state['step'])

            beta1, beta2, beta3 = group['betas']
            alpha = group['alpha']
            T_alpha_beta3 = group['T_alpha_beta3']

            self._update_adamemix(
                params_with_grad,
                grads,
                exp_avgs,
                exp_avg_sqs,
                exp_avg_slow,
                state_steps,
                beta1=beta1,
                beta2=beta2,
                beta3=beta3,
                alpha=alpha,
                T_alpha_beta3=T_alpha_beta3,
                lr=group['lr'],
                weight_decay=group['weight_decay'],
                eps=group['eps'],
            )

        return loss

    def _update_adamemix(self, params, grads, exp_avgs, exp_avg_sqs, exp_avg_slow, state_steps,
                         beta1, beta2, beta3, alpha, T_alpha_beta3, lr, weight_decay, eps):
        
        for i, param in enumerate(params):
            grad = grads[i]
            exp_avg = exp_avgs[i]
            exp_avg_sq = exp_avg_sqs[i]
            exp_avg_slow_i = exp_avg_slow[i]
            step = state_steps[i]

            bias_correction1 = 1 - beta1 ** step
            bias_correction2 = 1 - beta2 ** step

            if T_alpha_beta3 is not None:
                alpha_t = min(step * alpha / T_alpha_beta3, alpha)
                beta3_t = min(math.exp(math.log(beta1) * math.log(beta3) / 
                              ((1 - step / T_alpha_beta3) * math.log(beta3) + 
                               (step / T_alpha_beta3) * math.log(beta1))), beta3)
            else:
                alpha_t = alpha
                beta3_t = beta3

            # Decay the first and second moment running average coefficient
            exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
            exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)
            exp_avg_slow_i.mul_(beta3_t).add_(grad, alpha=1 - beta3_t)

            denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(eps)

            step_size = lr / bias_correction1

            if weight_decay != 0:
                param.add_(param, alpha=-weight_decay * lr)

            param.addcdiv_(exp_avg + alpha_t * exp_avg_slow_i, denom, value=-step_size)

In [51]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = torch.optim.AdamW(optimizer_grouped_parameters,
                        lr=2e-5)
# optimizer = AdEMAMix(optimizer_grouped_parameters,
#                         lr=3e-5,weight_decay = 0.1, alpha=8.0)

In [52]:
'''
Implementation of Dual Focal Loss.
Reference:
[1]  Tao, Linwei, Minjing Dong, and Chang Xu. "Dual Focal Loss for Calibration." arXiv preprint arXiv:2305.13665 (2023).
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class DualFocalLoss(nn.Module):
    def __init__(self, gamma=0, size_average=False):
        super(DualFocalLoss, self).__init__()
        self.gamma = gamma
        self.size_average = size_average


    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logp_k = F.log_softmax(input, dim=1)
        softmax_logits = logp_k.exp()
        logp_k = logp_k.gather(1, target)
        logp_k = logp_k.view(-1)
        p_k = logp_k.exp()  # p_k: probility at target label
        p_j_mask = torch.lt(softmax_logits, p_k.reshape(p_k.shape[0], 1)) * 1  # mask all logit larger and equal than p_k
        p_j = torch.topk(p_j_mask * softmax_logits, 1)[0].squeeze()

        loss = -1 * (1 - p_k + p_j) ** self.gamma * logp_k



        if self.size_average: return loss.mean()
        else: return loss.sum()

In [53]:
NUM_EPOCHS = 8

In [54]:
criterion = DualFocalLoss(gamma = 2)

In [55]:
num_training_step = len(train_loader)*NUM_EPOCHS
scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=num_training_step//10, num_training_steps=num_training_step)


In [56]:
tokenizer.convert_tokens_to_ids('<image>')

250002

In [57]:
from tqdm import tqdm

def train_one_epoch(model, criterion, dataloader, epoch,device,scaler):
    epoch_loss = 0

    model.train()

    # batch accumulation parameter
    accum_iter = 2

    with tqdm(dataloader, unit="batch") as tepoch:
        for i, data in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")

            input_ids, attention_mask, pixel_values, label = data
            pixel_values = pixel_values.float()
            
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            pixel_values = pixel_values.to(device)
            label = label.to(device)
            
            with torch.autocast(device_type='cuda', dtype=torch.bfloat16 if mode == 'BF16' else torch.float16, enabled=True if '16' in mode else False):
                logits = model(
                    vision_x =  pixel_values,
                    lang_x =  input_ids,
                    attention_mask = attention_mask
                    )
                losses = criterion(logits,label)
                
            lll = losses.item()
            epoch_loss += lll
            
            all_asp_loss = losses / accum_iter
            scaler.scale(all_asp_loss).backward()
            embed_grad = (
                    model.lang_encoder.get_input_embeddings().weight.grad
                        )
            zero_mask = torch.zeros_like(embed_grad)
            zero_mask[250002] = torch.ones_like(zero_mask[250002])
            model.lang_encoder.get_input_embeddings().weight.grad = (
                                embed_grad * zero_mask
            )

            if ((i + 1) % accum_iter == 0) or (i + 1 == len(dataloader)):
                torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()


            tepoch.set_postfix(loss=lll)

        epoch_loss /= i
        print(f"At EPOCH {epoch}, loss = {epoch_loss}")
        print("======================================")

    return epoch_loss

In [58]:
from sklearn.metrics import precision_recall_fscore_support

def macro_f1(y_true, y_pred):
    p_macro, r_macro, f_macro, support_macro \
      = precision_recall_fscore_support(y_true, y_pred, average=None,zero_division=0.0,labels = [0,1,2,3])
    return p_macro, r_macro, f_macro


In [59]:
llabels = ['not-sarcasm', 'image-sarcasm','text-sarcasm','multi-sarcasm']
def eval(model, criterion, dataloader,device):

    print("EVAL STEP: ")

    model.eval()
    epoch_loss = 0
    preds = []
    labels = []
    with tqdm(dataloader, unit="batch") as tepoch:
        for i, data in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")

            input_ids, attention_mask, pixel_values, label = data
            pixel_values = pixel_values.float()

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            pixel_values = pixel_values.to(device)
            label = label.to(device)

            with torch.no_grad():
                logits = model(
                    vision_x =  pixel_values,
                    lang_x =  input_ids,
                    attention_mask = attention_mask
                    )
                
                eval_losses = criterion(logits,label)
                pred = np.argmax(logits.cpu().detach(),axis = -1).to('cpu').numpy()
                label = label.to('cpu').numpy()
                
                preds.append(pred)
                labels.append(label)

            epoch_loss += eval_losses
            
        epoch_loss /= i
        print(f"Loss = {epoch_loss}")
        
        preds = np.concatenate(preds)
        labels = np.concatenate(labels)
        precision, recall, f1_score = macro_f1(labels,preds)
        for idx, ll in enumerate(llabels):
            print(f"{ll}:")
            print("\t- Precision:", precision[idx])
            print("\t- Recall:", recall[idx])
            print("\t- F1-score:", f1_score[idx])
            print("========================")
        print("===============NEXT STEP===============")

        return epoch_loss

In [60]:
def save_model(path, model, epoch):
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
    },path)

In [61]:
def load_model(path):
    check_point = torch.load(path)
    return check_point

In [62]:
best_loss = float('inf')
count = 0
for epoch in range(NUM_EPOCHS):
    train_one_epoch(model, criterion, train_loader, epoch, device,scaler)
    eval_loss = eval(model, criterion,dev_loader,device)
    if epoch >= NUM_EPOCHS - 3:
        save_model(f'model_{epoch}.pth',model, epoch)


Epoch 0: 100%|██████████| 2431/2431 [11:27<00:00,  3.54batch/s, loss=0.838] 


At EPOCH 0, loss = 2.2963897759286462
EVAL STEP: 


Epoch 0: 100%|██████████| 271/271 [00:46<00:00,  5.83batch/s]


Loss = 1.8356678485870361
not-sarcasm:
	- Precision: 0.7447154471544716
	- Recall: 0.7557755775577558
	- F1-score: 0.7502047502047502
image-sarcasm:
	- Precision: 0.0
	- Recall: 0.0
	- F1-score: 0.0
text-sarcasm:
	- Precision: 0.0
	- Recall: 0.0
	- F1-score: 0.0
multi-sarcasm:
	- Precision: 0.6180257510729614
	- Recall: 0.6808510638297872
	- F1-score: 0.6479190101237345
===============NEXT STEP===============


Epoch 1: 100%|██████████| 2431/2431 [11:13<00:00,  3.61batch/s, loss=1.52]  


...

Loss = 1.8175466060638428
not-sarcasm:
	- Precision: 0.831041257367387
	- Recall: 0.698019801980198
	- F1-score: 0.7587443946188339
image-sarcasm:
	- Precision: 0.625
	- Recall: 0.11363636363636363
	- F1-score: 0.1923076923076923
text-sarcasm:
	- Precision: 1.0
	- Recall: 0.125
	- F1-score: 0.2222222222222222
multi-sarcasm:
	- Precision: 0.6127886323268206
	- Recall: 0.8156028368794326
	- F1-score: 0.6997971602434077
===============NEXT STEP===============


Epoch 7: 100%|██████████| 2431/2431 [11:06<00:00,  3.65batch/s, loss=0.0805] 


At EPOCH 7, loss = 1.1836966464659122
EVAL STEP: 


Epoch 7: 100%|██████████| 271/271 [00:39<00:00,  6.88batch/s]


Loss = 2.0319113731384277
not-sarcasm:
	- Precision: 0.8067978533094812
	- Recall: 0.7442244224422442
	- F1-score: 0.7742489270386265
image-sarcasm:
	- Precision: 0.6666666666666666
	- Recall: 0.13636363636363635
	- F1-score: 0.22641509433962262
text-sarcasm:
	- Precision: 1.0
	- Recall: 0.125
	- F1-score: 0.2222222222222222
multi-sarcasm:
	- Precision: 0.630859375
	- Recall: 0.7635933806146572
	- F1-score: 0.6909090909090908
===============NEXT STEP===============


# MAKE RESULT

In [ ]:
test_data = pd.read_json('kaggle/input/dev-dsc2024/vimmsd-public-test.json',orient = 'index').reset_index()

In [ ]:
normalize_class = TextNormalize()
test_data['caption'] = test_data['caption'].apply(lambda x: normalize_class.normalize(text_normalize(convert_unicode(x))))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_PATH)
tokenizer.add_special_tokens(
        {"additional_special_tokens": ["<image>"]}
    )

# PHASE 1

In [ ]:
class TestDS_1(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, img_folder, vision_processor, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.img_folder = img_folder
        self.vision_processor = vision_processor
        self.labels = ['not-sarcasm','sarcasm']

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        text = self.data.loc[idx,'caption']
        
        index = self.data.loc[idx,'index']
        img_name = self.data.loc[idx,'image']
        img_file = os.path.join(self.img_folder,img_name)
        
        tokenizer_seq_dict = self.tokenizer(f"<image> {text}", max_length=self.max_len,truncation=True,padding='max_length', return_length=True, return_tensors='pt')
        input_ids = tokenizer_seq_dict['input_ids'][0]
        attention_mask = torch.zeros(input_ids.size())

        image = Image.open(img_file)
        pixel_values = self.vision_processor(images=image, return_tensors="pt")['pixel_values'].unsqueeze(1)
        
        return input_ids, attention_mask, pixel_values, index   

In [ ]:
vision_processor = CLIPProcessor.from_pretrained('openai/clip-vit-large-patch14')

In [ ]:
test_set = TestDS_1(test_data, tokenizer, '/kaggle/input/dev-dsc2024/public-test-images/dev-images', vision_processor, max_len = 150)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 4, shuffle=True)

In [ ]:
lang_encoder = AutoModel.from_pretrained('susnato/ernie-m-base_pytorch')
vision_encoder = CLIPVisionModel.from_pretrained('openai/clip-vit-large-patch14')

extend_instance(lang_encoder, FlamingoLMMixin)
encoder_layers_attr_name  = _infer_encoder_layers_attr_name(lang_encoder)
lang_encoder.set_encoder_layers_attr_name(encoder_layers_attr_name)
lang_encoder.resize_token_embeddings(len(tokenizer))


In [ ]:
model_1 = Flamingo(
    vision_encoder,
    lang_encoder,
    cross_attn_every_n_layers=1,
    vis_dim = 1024,
    media_token_id = tokenizer.convert_tokens_to_ids('<image>'),
    n_classes = 1,
    gradient_checkpointing=False
)

In [ ]:
model_1.to(device)

In [ ]:
checkpoint = load_model('phase1_7.pth')
model_1.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
labels = test_set.labels

In [ ]:
results = {}
for data in tqdm(test_loader):
    
    input_ids, attention_mask, pixel_values, indexes = data
    pixel_values = pixel_values.float()

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    pixel_values = pixel_values.to(device)

    with torch.no_grad():
        logits = model_1(
            vision_x =  pixel_values,
            lang_x =  input_ids,
            )
        pred = (torch.nn.functional.sigmoid(logits) >= 0.6).to(torch.int).cpu().detach().numpy().reshape(-1)
        
    for idx, pred in zip(indexes,pred):
        results[idx.item()] = labels[pred]


In [ ]:
results

In [ ]:
final_results = {
    'results': results,
    'phase': 'dev'
}
len(final_results['results'])

In [ ]:
with open('phase_1_results.json', 'w',encoding = 'utf-8') as f:
    json.dump(final_results, f, ensure_ascii = False, indent = 2)

# PHASE 2

In [ ]:
class TestDS(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, img_folder, vision_processor, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.img_folder = img_folder
        self.vision_processor = vision_processor
        self.labels = ['image-sarcasm','text-sarcasm','multi-sarcasm']

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        text = self.data.loc[idx,'caption']
        
        index = self.data.loc[idx,'index']
        img_name = self.data.loc[idx,'image']
        img_file = os.path.join(self.img_folder,img_name)
        
        tokenizer_seq_dict = self.tokenizer(f"<image> {text}", max_length=self.max_len,truncation=True,padding='max_length', return_length=True, return_tensors='pt')
        input_ids = tokenizer_seq_dict['input_ids'][0]
        attention_mask = torch.zeros(input_ids.size())

        image = read_image(img_file).permute(1,2,0)[:,:,:3]
        pixel_values = self.vision_processor(images=image, return_tensors="pt")['pixel_values'].unsqueeze(1)
        
        return input_ids, attention_mask, pixel_values, index   

In [ ]:
test_data_2 = pd.read_json('phase_1_results.json').reset_index()
test_data_2.shape

In [ ]:
test_data_2 = test_data_2[test_data_2['results'] == 'sarcasm']
test_data_2.shape

In [ ]:
test_data_2 = test_data[test_data['index'].isin(test_data_2['index'].values.tolist())].reset_index()
test_data_2

In [ ]:
test_set = TestDS(test_data_2, tokenizer, '/kaggle/input/dev-dsc2024/public-test-images/dev-images', vision_processor, max_len = 200)

In [ ]:
test_set[0]

In [ ]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 4, shuffle=True)

In [ ]:
VISION_PRETRAINED_PATH

In [ ]:
lang_encoder = AutoModel.from_pretrained(PRETRAINED_PATH)
vision_encoder = AutoModel.from_pretrained(VISION_PRETRAINED_PATH)

extend_instance(lang_encoder, FlamingoLMMixin)
encoder_layers_attr_name  = _infer_encoder_layers_attr_name(lang_encoder)
lang_encoder.set_encoder_layers_attr_name(encoder_layers_attr_name)
lang_encoder.resize_token_embeddings(len(tokenizer))


In [ ]:
model = Flamingo(
    vision_encoder,
    lang_encoder,
    cross_attn_every_n_layers=1,
    vis_dim = 1024,
    media_token_id = tokenizer.convert_tokens_to_ids('<image>'),
    gradient_checkpointing=False,
    n_classes = 3
)
model.to(device)

In [ ]:
checkpoint = load_model('model_4.pth')
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
labels = test_set.labels
labels

In [ ]:
results = {}
for data in tqdm(test_loader):
    
    input_ids, attention_mask, pixel_values, indexes = data
    pixel_values = pixel_values.float()

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    pixel_values = pixel_values.to(device)

    with torch.no_grad():
        logits = model(
            vision_x =  pixel_values,
            lang_x =  input_ids,
            )
        pred = np.argmax(logits.cpu().detach(),axis = -1).to('cpu').numpy()
    
    for idx, pred in zip(indexes,pred):
        results[idx.item()] = labels[pred]

In [ ]:
final_results = {
    'results': results,
    'phase': 'dev'
}
len(final_results['results'])

In [ ]:
final_results

In [ ]:
with open('phase_2_results.json', 'w',encoding = 'utf-8') as f:
    json.dump(final_results, f, ensure_ascii = False, indent = 2)

In [ ]:
with open('phase_1_results.json', 'r',encoding = 'utf-8') as f:
    phase_1_results = json.load(f)

In [ ]:
with open('phase_2_results.json', 'r',encoding = 'utf-8') as f:
    phase_2_results = json.load(f)

In [ ]:
final = phase_1_results.copy()
len(final['results'].items())

In [ ]:
for k,v in phase_2_results['results'].items():
    final['results'][k] = v

In [ ]:
len(final['results'].items())

In [ ]:
with open('results.json', 'w',encoding = 'utf-8') as f:
    json.dump(final, f, ensure_ascii = False, indent = 2)